In [1]:
from sklearn.model_selection import StratifiedKFold, train_test_split
import pandas as pd
pd.set_option('display.max_columns',100)
import os
import seaborn as sns
sns.set()
import numpy as np
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
from sklearn.ensemble import AdaBoostClassifier,GradientBoostingClassifier
plt.rcParams['figure.figsize'] = 8, 5
plt.style.use("fivethirtyeight")
for dirname, _, filenames in os.walk('Predicting_Coupon_Redemption'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [2]:
train_df = pd.read_csv('../Data/train.csv')
campaign_data_df = pd.read_csv('../Data/campaign_data.csv')
coupon_item_mapping_df = pd.read_csv('../Data/coupon_item_mapping.csv')
customer_demographics_df = pd.read_csv('../Data/customer_demographics.csv')
customer_transaction_data_df = pd.read_csv('../Data/customer_transaction_data.csv')
item_data_df = pd.read_csv('../Data/item_data.csv')
test_df = pd.read_csv('../Data/test.csv')
sub_df = pd.read_csv('../Data/sample_submission.csv')




In [3]:
# Feature Engineering in campaign_data.csv data.
campaign_data_df['start_date'] = pd.to_datetime(campaign_data_df['start_date'], format='%d/%m/%y', dayfirst=True)
campaign_data_df['end_date'] = pd.to_datetime(campaign_data_df['end_date'], format='%d/%m/%y', dayfirst=True)
campaign_data_df['diff_d'] = (campaign_data_df['end_date'] - campaign_data_df['start_date']) / np.timedelta64(1,'D')
campaign_data_df.drop(['start_date','end_date'], axis=1, inplace=True)

No Feature Engineering required for the train.csv data.

No Feature Engineering required for the item_data.csv data.

No Feature Engineering required for the coupon_item_mapping.csv data.

Feature Engineering in customer_demographics.csv data.

In [4]:
lb = LabelEncoder()
customer_demographics_df['age_range'] = lb.fit_transform(customer_demographics_df['age_range'])

In [5]:
# customer_transaction_data.csv
customer_transaction_data_df['date'] = pd.to_datetime(customer_transaction_data_df['date'], format='%Y-%m-%d')
customer_transaction_data_df['date_d'] = customer_transaction_data_df['date'].dt.day.astype('category')
customer_transaction_data_df['date_m'] = customer_transaction_data_df['date'].dt.month.astype('category')
customer_transaction_data_df['date_w'] = customer_transaction_data_df['date'].dt.week.astype('category')
customer_transaction_data_df.drop(['date'], axis=1, inplace=True)

In [6]:
tgroup = customer_transaction_data_df.groupby(['customer_id']).sum().reset_index()

In [7]:
# Merge Data.
train_campaign_data = pd.merge(train_df,campaign_data_df,on='campaign_id',how="left")
test_campaign_data= pd.merge(test_df,campaign_data_df,on='campaign_id',how="left")

In [8]:
# Coupon Item Mapping.
coupon_item_mapping_item_data = pd.merge(coupon_item_mapping_df, item_data_df, on='item_id', how="left")
mci_group = pd.DataFrame()
mci_group[['coupon_id','no_of_items']] = coupon_item_mapping_item_data.groupby('coupon_id').count().reset_index()[
    ['coupon_id','item_id']]
mci_group[['brand_type','category']] = coupon_item_mapping_item_data.groupby('coupon_id').max().reset_index()[
    ['brand_type','category']]


In [9]:
mdtg = pd.merge(tgroup,customer_demographics_df,on='customer_id',how='outer')


In [10]:
# Merge all.
mergeddata = pd.merge(train_campaign_data, mdtg, on=['customer_id'], how='left')
mergeddata = pd.merge(mergeddata, mci_group, on=['coupon_id'], how='left')

In [11]:
mergeddata2 = pd.merge(test_campaign_data, mdtg, on=['customer_id'], how='left')
mergeddata2 = pd.merge(mergeddata2, mci_group, on=['coupon_id'], how='left')
id_df = mergeddata2['id']
# mergeddata.drop(['id'],axis=1,inplace=True)
# mergeddata2.drop(['id'],axis=1,inplace=True)



In [12]:
print(mergeddata['redemption_status'].value_counts())
print(mergeddata['redemption_status'].value_counts(normalize=True))

0    77640
1      729
Name: redemption_status, dtype: int64
0    0.990698
1    0.009302
Name: redemption_status, dtype: float64


In [13]:
# Checking Missing values.
print(mergeddata.isnull().sum())
percent_missing = mergeddata.isnull().sum() * 100 / len(mergeddata)
missing_value_df = pd.DataFrame({'column_name': mergeddata.columns,
                                 'percent_missing': percent_missing})
print(missing_value_df)
mergeddata.drop(['no_of_children','age_range','marital_status','rented','family_size','income_bracket'], axis=1, inplace=True)
print(mergeddata.isnull().sum())

id                       0
campaign_id              0
coupon_id                0
customer_id              0
redemption_status        0
campaign_type            0
diff_d                   0
item_id                  0
quantity                 0
selling_price            0
other_discount           0
coupon_discount          0
age_range            34708
marital_status       52975
rented               34708
family_size          34708
no_of_children       64313
income_bracket       34708
no_of_items              0
brand_type               0
category                 0
dtype: int64
                         column_name  percent_missing
id                                id         0.000000
campaign_id              campaign_id         0.000000
coupon_id                  coupon_id         0.000000
customer_id              customer_id         0.000000
redemption_status  redemption_status         0.000000
campaign_type          campaign_type         0.000000
diff_d                        diff_d      

In [14]:
# Checking Missing values.
print(mergeddata2.isnull().sum())
percent_missing = mergeddata2.isnull().sum() * 100 / len(mergeddata2)
missing_value_df = pd.DataFrame({'column_name': mergeddata2.columns,
                                 'percent_missing': percent_missing})
print(missing_value_df)
mergeddata2.drop(['no_of_children','age_range','marital_status','rented','family_size','income_bracket'], axis=1, inplace=True)
print(mergeddata2.isnull().sum())

id                     0
campaign_id            0
coupon_id              0
customer_id            0
campaign_type          0
diff_d                 0
item_id                0
quantity               0
selling_price          0
other_discount         0
coupon_discount        0
age_range          19287
marital_status     32144
rented             19287
family_size        19287
no_of_children     40064
income_bracket     19287
no_of_items            0
brand_type             0
category               0
dtype: int64
                     column_name  percent_missing
id                            id         0.000000
campaign_id          campaign_id         0.000000
coupon_id              coupon_id         0.000000
customer_id          customer_id         0.000000
campaign_type      campaign_type         0.000000
diff_d                    diff_d         0.000000
item_id                  item_id         0.000000
quantity                quantity         0.000000
selling_price      selling_price     

In [15]:
# Feature Encoding.
cols = ['campaign_type','brand_type','category']
lb = LabelEncoder()
for i in cols:
    mergeddata[i] = lb.fit_transform(mergeddata[i])

In [16]:
for i in cols:
    mergeddata2[i] = lb.fit_transform(mergeddata2[i])


In [17]:
X = mergeddata.drop(['redemption_status'],axis=1)
Y = mergeddata['redemption_status']

In [18]:
# Handling class imbalance.
# Technique - 1.
fold = StratifiedKFold(n_splits=10, shuffle=True, random_state=2020)
i = 1
auc = []

for train_index, test_index in fold.split(X, Y):
    x_train, x_val = X.iloc[train_index], X.iloc[test_index]
    y_train, y_val = Y.iloc[train_index], Y.iloc[test_index]
    scaler = StandardScaler()
    x_train = scaler.fit_transform(x_train)
    x_val = scaler.transform(x_val)
    mergeddata2 = scaler.transform(mergeddata2)
    m = LGBMClassifier(random_state=80) # 93.23
    m.fit(x_train, y_train)
    pred_prob1 = m.predict_proba(x_val)
    auc.append(roc_auc_score(y_val, pred_prob1[:, 1]))
    i = i + 1

print("AUC Score")
print(sum(auc)/10)

AUC Score
0.927883427257051


In [19]:
# Technique-2:SMOTE
# Oversample the dataset.

from imblearn.over_sampling import SMOTE
from collections import Counter
oversample = SMOTE()
X,Y = oversample.fit_resample(X,Y)
counter = Counter(Y)
print(counter)

Counter({0: 77640, 1: 77640})


In [20]:
X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size=0.2,random_state = 101)

In [21]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [22]:
classifier_o = AdaBoostClassifier(random_state=20)  # 98.11
classifier_o.fit(X_train,y_train)
pred_prob1 = classifier_o.predict_proba(X_test)
print(roc_auc_score(y_test, pred_prob1[:, 1]))

0.9822363278225348


In [23]:
classifier_o = GradientBoostingClassifier(random_state=30)  # 98.74
classifier_o.fit(X_train,y_train)
pred_prob1 = classifier_o.predict_proba(X_test)
print(roc_auc_score(y_test, pred_prob1[:, 1]))

0.9876990018332462


In [24]:
classifier_o = LGBMClassifier(random_state=10)  # 99.78
classifier_o.fit(X_train,y_train)
pred_prob1 = classifier_o.predict_proba(X_test)
print(roc_auc_score(y_test,pred_prob1[:,1]))

0.9981574221524775


In [25]:
# LGBM Hyperparameter Tuning using Grid Search.
from sklearn.model_selection import GridSearchCV
param_grid = {'learning_rate': [0.1,0.5],
                 'max_depth': [4,5,6],
                 'num_leaves': [10,20],
                 'feature_fraction': [0.6,0.8],
                 'subsample': [0.2,0.6],
                 'objective': ['binary'],
              'metric': ['auc'],
              'is_unbalance':[False],
              'boosting':['gbdt'],
              'num_boost_round':[100],
              'early_stopping_rounds':[30]}

# Build and fit the GridSearchCV.
grid = GridSearchCV(estimator=classifier_o, param_grid=param_grid,
                    cv=10,verbose=10)
grid_results = grid.fit(X_train,y_train,eval_set = (X_test,y_test))
# Summarize the results in a readable format.
print("Best: {0}, using {1}".format(grid_results.best_score_, grid_results.best_params_))

Fitting 10 folds for each of 48 candidates, totalling 480 fits
[CV 1/10; 1/48] START boosting=gbdt, early_stopping_rounds=30, feature_fraction=0.6, is_unbalance=False, learning_rate=0.1, max_depth=4, metric=auc, num_boost_round=100, num_leaves=10, objective=binary, subsample=0.2
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] early_stopping_round is set=30, early_stopping_rounds=30 will be ignored. Current value: early_stopping_round=30
[LightGBM] [Warning] num_iterations is set=100, num_boost_round=100 will be ignored. Current value: num_iterations=100
[1]	valid_0's auc: 0.897
Training until validation scores don't improve for 30 rounds
[2]	valid_0's auc: 0.954588
[3]	valid_0's auc: 0.957
[4]	valid_0's auc: 0.959053
[5]	valid_0's auc: 0.95956
[6]	valid_0's auc: 0.961516
[7]	valid_0's

[1]	valid_0's auc: 0.896717
Training until validation scores don't improve for 30 rounds
[2]	valid_0's auc: 0.954283
[3]	valid_0's auc: 0.95713
[4]	valid_0's auc: 0.959547
[5]	valid_0's auc: 0.959902
[6]	valid_0's auc: 0.961913
[7]	valid_0's auc: 0.96235
[8]	valid_0's auc: 0.962153
[9]	valid_0's auc: 0.962828
[10]	valid_0's auc: 0.963364
[11]	valid_0's auc: 0.963647
[12]	valid_0's auc: 0.96413
[13]	valid_0's auc: 0.963831
[14]	valid_0's auc: 0.965364
[15]	valid_0's auc: 0.966004
[16]	valid_0's auc: 0.966984
[17]	valid_0's auc: 0.968177
[18]	valid_0's auc: 0.969379
[19]	valid_0's auc: 0.969696
[20]	valid_0's auc: 0.969886
[21]	valid_0's auc: 0.970433
[22]	valid_0's auc: 0.971246
[23]	valid_0's auc: 0.97159
[24]	valid_0's auc: 0.971954
[25]	valid_0's auc: 0.97245
[26]	valid_0's auc: 0.973123
[27]	valid_0's auc: 0.973862
[28]	valid_0's auc: 0.974135
[29]	valid_0's auc: 0.974372
[30]	valid_0's auc: 0.974551
[31]	valid_0's auc: 0.975098
[32]	valid_0's auc: 0.976018
[33]	valid_0's auc: 0.977

[49]	valid_0's auc: 0.981729
[50]	valid_0's auc: 0.981809
[51]	valid_0's auc: 0.982334
[52]	valid_0's auc: 0.982713
[53]	valid_0's auc: 0.982918
[54]	valid_0's auc: 0.983205
[55]	valid_0's auc: 0.98342
[56]	valid_0's auc: 0.983579
[57]	valid_0's auc: 0.983737
[58]	valid_0's auc: 0.983906
[59]	valid_0's auc: 0.983971
[60]	valid_0's auc: 0.984517
[61]	valid_0's auc: 0.984627
[62]	valid_0's auc: 0.985073
[63]	valid_0's auc: 0.985523
[64]	valid_0's auc: 0.985637
[65]	valid_0's auc: 0.985746
[66]	valid_0's auc: 0.985972
[67]	valid_0's auc: 0.986094
[68]	valid_0's auc: 0.986276
[69]	valid_0's auc: 0.986389
[70]	valid_0's auc: 0.986482
[71]	valid_0's auc: 0.986888
[72]	valid_0's auc: 0.987329
[73]	valid_0's auc: 0.987401
[74]	valid_0's auc: 0.987786
[75]	valid_0's auc: 0.988146
[76]	valid_0's auc: 0.988271
[77]	valid_0's auc: 0.988317
[78]	valid_0's auc: 0.988372
[79]	valid_0's auc: 0.988494
[80]	valid_0's auc: 0.988634
[81]	valid_0's auc: 0.988671
[82]	valid_0's auc: 0.988923
[83]	valid_0's 

[96]	valid_0's auc: 0.990248
[97]	valid_0's auc: 0.990333
[98]	valid_0's auc: 0.990365
[99]	valid_0's auc: 0.990488
[100]	valid_0's auc: 0.990521
Did not meet early stopping. Best iteration is:
[100]	valid_0's auc: 0.990521
[CV 7/10; 1/48] END boosting=gbdt, early_stopping_rounds=30, feature_fraction=0.6, is_unbalance=False, learning_rate=0.1, max_depth=4, metric=auc, num_boost_round=100, num_leaves=10, objective=binary, subsample=0.2; total time=   0.5s
[CV 8/10; 1/48] START boosting=gbdt, early_stopping_rounds=30, feature_fraction=0.6, is_unbalance=False, learning_rate=0.1, max_depth=4, metric=auc, num_boost_round=100, num_leaves=10, objective=binary, subsample=0.2
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] early_stopping_round is set=30, early_stopping_rounds=30 will be ignore

[1]	valid_0's auc: 0.896723
Training until validation scores don't improve for 30 rounds
[2]	valid_0's auc: 0.954374
[3]	valid_0's auc: 0.956665
[4]	valid_0's auc: 0.959492
[5]	valid_0's auc: 0.957797
[6]	valid_0's auc: 0.959578
[7]	valid_0's auc: 0.95987
[8]	valid_0's auc: 0.963221
[9]	valid_0's auc: 0.963786
[10]	valid_0's auc: 0.96389
[11]	valid_0's auc: 0.96412
[12]	valid_0's auc: 0.965
[13]	valid_0's auc: 0.964464
[14]	valid_0's auc: 0.965731
[15]	valid_0's auc: 0.966215
[16]	valid_0's auc: 0.9671
[17]	valid_0's auc: 0.968332
[18]	valid_0's auc: 0.969545
[19]	valid_0's auc: 0.970006
[20]	valid_0's auc: 0.970068
[21]	valid_0's auc: 0.970419
[22]	valid_0's auc: 0.971251
[23]	valid_0's auc: 0.971682
[24]	valid_0's auc: 0.971953
[25]	valid_0's auc: 0.972574
[26]	valid_0's auc: 0.973081
[27]	valid_0's auc: 0.973273
[28]	valid_0's auc: 0.973595
[29]	valid_0's auc: 0.973733
[30]	valid_0's auc: 0.974099
[31]	valid_0's auc: 0.974232
[32]	valid_0's auc: 0.975347
[33]	valid_0's auc: 0.976425

...

[67]	valid_0's auc: 0.998837
[68]	valid_0's auc: 0.998851
[69]	valid_0's auc: 0.998861
[70]	valid_0's auc: 0.998722
[71]	valid_0's auc: 0.998772
[72]	valid_0's auc: 0.998806
[73]	valid_0's auc: 0.997925
[74]	valid_0's auc: 0.998647
[75]	valid_0's auc: 0.99873
[76]	valid_0's auc: 0.998695
[77]	valid_0's auc: 0.998727
[78]	valid_0's auc: 0.997949
[79]	valid_0's auc: 0.998678
[80]	valid_0's auc: 0.998552
[81]	valid_0's auc: 0.998689
[82]	valid_0's auc: 0.998495
[83]	valid_0's auc: 0.998687
[84]	valid_0's auc: 0.998713
[85]	valid_0's auc: 0.998731
[86]	valid_0's auc: 0.998737
[87]	valid_0's auc: 0.998735
[88]	valid_0's auc: 0.998756
[89]	valid_0's auc: 0.998773
[90]	valid_0's auc: 0.998778
[91]	valid_0's auc: 0.998785
[92]	valid_0's auc: 0.998803
[93]	valid_0's auc: 0.998805
[94]	valid_0's auc: 0.998837
[95]	valid_0's auc: 0.998851
[96]	valid_0's auc: 0.998865
[97]	valid_0's auc: 0.998872
[98]	valid_0's auc: 0.998882
[99]	valid_0's auc: 0.998885
[100]	valid_0's auc: 0.99889
Did not meet ea

[CV 4/10; 48/48] END boosting=gbdt, early_stopping_rounds=30, feature_fraction=0.8, is_unbalance=False, learning_rate=0.5, max_depth=6, metric=auc, num_boost_round=100, num_leaves=20, objective=binary, subsample=0.6; total time=   0.7s
[CV 5/10; 48/48] START boosting=gbdt, early_stopping_rounds=30, feature_fraction=0.8, is_unbalance=False, learning_rate=0.5, max_depth=6, metric=auc, num_boost_round=100, num_leaves=20, objective=binary, subsample=0.6
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] early_stopping_round is set=30, early_stopping_rounds=30 will be ignored. Current value: early_stopping_round=30
[LightGBM] [Warning] num_iterations is set=100, num_boost_round=100 will be ignored. Current value: num_iterations=100
[1]	valid_0's auc: 0.946933
Training until validation scores 

[1]	valid_0's auc: 0.947225
Training until validation scores don't improve for 30 rounds
[2]	valid_0's auc: 0.962403
[3]	valid_0's auc: 0.965777
[4]	valid_0's auc: 0.970901
[5]	valid_0's auc: 0.976792
[6]	valid_0's auc: 0.97984
[7]	valid_0's auc: 0.982101
[8]	valid_0's auc: 0.986726
[9]	valid_0's auc: 0.988551
[10]	valid_0's auc: 0.988894
[11]	valid_0's auc: 0.989649
[12]	valid_0's auc: 0.991203
[13]	valid_0's auc: 0.991513
[14]	valid_0's auc: 0.992422
[15]	valid_0's auc: 0.992934
[16]	valid_0's auc: 0.993544
[17]	valid_0's auc: 0.993923
[18]	valid_0's auc: 0.994374
[19]	valid_0's auc: 0.994785
[20]	valid_0's auc: 0.995148
[21]	valid_0's auc: 0.99533
[22]	valid_0's auc: 0.995558
[23]	valid_0's auc: 0.995873
[24]	valid_0's auc: 0.995962
[25]	valid_0's auc: 0.996118
[26]	valid_0's auc: 0.996345
[27]	valid_0's auc: 0.996505
[28]	valid_0's auc: 0.996756
[29]	valid_0's auc: 0.996964
[30]	valid_0's auc: 0.997124
[31]	valid_0's auc: 0.997271
[32]	valid_0's auc: 0.997399
[33]	valid_0's auc: 0.

[34]	valid_0's auc: 0.997392
[35]	valid_0's auc: 0.997507
[36]	valid_0's auc: 0.997532
[37]	valid_0's auc: 0.997582
[38]	valid_0's auc: 0.997838
[39]	valid_0's auc: 0.997959
[40]	valid_0's auc: 0.997989
[41]	valid_0's auc: 0.998101
[42]	valid_0's auc: 0.998151
[43]	valid_0's auc: 0.998195
[44]	valid_0's auc: 0.998258
[45]	valid_0's auc: 0.998224
[46]	valid_0's auc: 0.998289
[47]	valid_0's auc: 0.998329
[48]	valid_0's auc: 0.998352
[49]	valid_0's auc: 0.998436
[50]	valid_0's auc: 0.998484
[51]	valid_0's auc: 0.998521
[52]	valid_0's auc: 0.99855
[53]	valid_0's auc: 0.998569
[54]	valid_0's auc: 0.998619
[55]	valid_0's auc: 0.998658
[56]	valid_0's auc: 0.998691
[57]	valid_0's auc: 0.99872
[58]	valid_0's auc: 0.998763
[59]	valid_0's auc: 0.998823
[60]	valid_0's auc: 0.99887
[61]	valid_0's auc: 0.998874
[62]	valid_0's auc: 0.998904
[63]	valid_0's auc: 0.99893
[64]	valid_0's auc: 0.998946
[65]	valid_0's auc: 0.998952
[66]	valid_0's auc: 0.998901
[67]	valid_0's auc: 0.998946
[68]	valid_0's auc

[63]	valid_0's auc: 0.998548
[64]	valid_0's auc: 0.998579
[65]	valid_0's auc: 0.998614
[66]	valid_0's auc: 0.99867
[67]	valid_0's auc: 0.998673
[68]	valid_0's auc: 0.998694
[69]	valid_0's auc: 0.998709
[70]	valid_0's auc: 0.998734
[71]	valid_0's auc: 0.998775
[72]	valid_0's auc: 0.998813
[73]	valid_0's auc: 0.998812
[74]	valid_0's auc: 0.998841
[75]	valid_0's auc: 0.998868
[76]	valid_0's auc: 0.998889
[77]	valid_0's auc: 0.998891
[78]	valid_0's auc: 0.998074
[79]	valid_0's auc: 0.998788
[80]	valid_0's auc: 0.998795
[81]	valid_0's auc: 0.998812
[82]	valid_0's auc: 0.99882
[83]	valid_0's auc: 0.998819
[84]	valid_0's auc: 0.998832
[85]	valid_0's auc: 0.998836
[86]	valid_0's auc: 0.998843
[87]	valid_0's auc: 0.998863
[88]	valid_0's auc: 0.996649
[89]	valid_0's auc: 0.998344
[90]	valid_0's auc: 0.997901
[91]	valid_0's auc: 0.99847
[92]	valid_0's auc: 0.997443
[93]	valid_0's auc: 0.997576
[94]	valid_0's auc: 0.998348
[95]	valid_0's auc: 0.993345
[96]	valid_0's auc: 0.99856
[97]	valid_0's auc

In [26]:
classifier_o = LGBMClassifier(random_state=10,boosting='gbdt',feature_fraction=0.8,is_unbalance=False,learning_rate=0.5,max_depth=6,metric='auc',num_boost_round=100,num_leaves=20,objective='binary',subsample=0.2)  # 99.78
classifier_o.fit(X_train,y_train)
pred_prob1 = classifier_o.predict_proba(X_test)
print(roc_auc_score(y_test,pred_prob1[:,1]))  # 99.89

0.9985087568960326


In [27]:
pred = classifier_o.predict(mergeddata2)
sub_df['redemption_status'] = pred
sub_df.to_csv('final_predictions.csv', index=False)